## Энергетический оракул
Ноутбук команды #12

Работа выполнена на основе модели LightGBM


### 1. Подготовка данных

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import random

import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

import re

from tqdm import tqdm
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True )


random_state = 12345
NUM_ITERATIONS = 5000

FEATURES = 'base feature'

#### 1.1 Функции для расшифровки прогноза погоды в колонке 'weather_pred'

In [ ]:
# Расшифровка прогноза в колонке 'weather_pred'

# функция формирует колонки 'cloudy', 'rainy', 'windy', 'clear', 'rain_probability', 'has_rain_probability'
# в колонках число, которое 0 при отсутсвии упоминания явления в weather_pred или степень упоминания
# функция дает в колонках номер первого списка, элемент которого есть в строке плюс 1
# списки cloudy_list, rainy_list, windy_list, clear_list можно модифицировать
# соответственно, можно экспериментировать с расположением значений в списках
# например, сейчас 'дождь', 'снег', 'д+сн' - первая степень  дождя, а 'гроз', 'ливень' - вторая
# а можно сделать снег второй, а грозу с ливнем убрать в третью
# также сделал отдельный список для "ясности", чтобы выделить 'ясно' и 'солнечно'

def in_what_list(weather, big_list):
    for list_number, small_list in enumerate(big_list):
        if any(word in weather for word in small_list):
            return list_number+1
    return 0

def weather_split2(row):
    weather = row['weather_pred']
    cloudy_list = [['проясн', 'пер.об.', 'п/об'], ['пасм', 'обл']]
    rainy_list = [['дождь', 'снег', 'д+сн'], ['гроз', 'ливень']]
    windy_list = [['вет'],['штор']]
    clear_list = [['проясн'], ['ясно'], ['солнеч']]
    numbers = re.findall(r'\d+', weather)
    cloudy = in_what_list(weather, cloudy_list)
    rainy = in_what_list(weather, rainy_list)
    windy = in_what_list(weather, windy_list)
    clear = in_what_list(weather, clear_list)
    rain_probability = 0 if len(numbers)==0 else int(numbers[0])
    has_rain_probability = int(len(numbers)==0)
    return cloudy, rainy, windy, clear, rain_probability, has_rain_probability

def fill_weather_columns(df):
    df['weather_pred'] = df['weather_pred'].fillna('')
    df['cloudy'], df['rainy'], df['windy'], df['clear'], df['rain_probability'], df['has_rain_probability'] = \
                zip(*df.apply(weather_split2, axis=1))
    return df

#### 1.2 Функции для загрузки данных о ВВП 
данные загружаются из файла 'data/VVP.csv'

Некоторые научные работы указывают на прямую связь величины потребления электричества и показателя ВВП, который отражает ситуацию в экономике. Данные по экономике публикуются различными министерствами с разной периодичностью. Для использования в работе были взяты фактические данные по ВВП с сайта investing, который агрегирует публикации Минэкономразвития. Данные за месяц побликуются с месячной задержкой, поэтому модель использует для прогнозирования данные за прошлые месяцы, которые известны.   
  
Ссылка на данные: https://ru.investing.com/economic-calendar/russian-monthly-gdp-407


In [ ]:
# Функция добавляет данные о ВВП из файла 'data/VVP.csv' в датасет

def add_vvp2(data, file_source = 'data/VVP.csv'):
    """
    сырой датафрем подаем на вход
    """
    # обработаем файл с динамикой ВВП
    vvp = pd.read_csv(file_source)
    # преобразуем дату файла-источника в формат datetime64 и дропнем один столбик
    vvp['date'] = pd.to_datetime(vvp['date'], format ='%Y-%m-%d %H:%M:%S')
    vvp.drop('for_month',axis=1,inplace=True) 
    
    # обработаем основной фрейм - создадим столбец для соединения, который потом удалим
    data['date_temp'] = pd.to_datetime(data['date'], format = '%Y-%m-%d' )
    data['date_temp'] = data['date_temp'] + pd.to_timedelta(data['time'] , 'H')
    
    # соединяем основной фрейм и ВВП по дате объявления показтеля ВВП
    for idx in reversed(vvp.index):
        data.loc[data['date_temp']>=vvp.date[idx],'VVP'] = vvp.VVP_perc[idx]
        
    data.drop('date_temp',axis=1,inplace=True)   

    return data

#### 1.3 Функции для загрузки архива данных о фактической погоде
данные загружаются из файла 'data/preprocessing_loaded_table.csv'

Изначально данные для формирования таблицы "preprocessing_loaded_table" были взяты из с сайта [https://rp5.ru](https://rp5.ru/Архив_погоды_в_Храброво,_им._императрицы_Елизаветы_Петровны_(аэропорт),_METAR), где хранятся архивы погоды в аэрапорту Калининграда, за период с 31.12.2018 по 30.09.2023

Описание данных в таблице:
- Местное время в Храброво / им. императрицы Елизаветы Петровны (аэропорт) - Дата / Местное время
- T -  Темпиратура воздуха
- Po - Давление на уровне станции
- P - Давление приведённое к уровню моря
- U - Относительная влажность
- DD - Направление ветра
- Ff - Скорость ветра
- ff10 - Максимальное значение порыва ветра
- WW - Особое явление текущей погоды (осадки)
- W'W' - Явление недавней погоды, имеющее оперативное значение
- с - Общая облачность
- VV - Горизонтальная дальность видимости
- Td - Темпиратура точки росы

Данные, которые были взяты из данной таблицы и загружаются из 'data/preprocessing_loaded_table.csv':
- P - не подверглось изменению
- U - не подверглось изменению
- Td - не подверглась изменению

 WW - разделили на 4 категории:
- Нет осадков (где были пропуски)
- слабый дождь
- сильный дождь
- снег

DD - создали 4 столбца, соответствующих сторонам горизонта, которые принимали значения 0; 0.5 и 1 в зависимости от силы ветра в конкретном направлении
- N - north
- S - south
- W - west
- E - east

В дальнейшем эти данные использовались с лагом в сутки: в поля на завтрашний день записывались данные сегодняшнего.

In [ ]:
# Функции для работы с данными о фактической погоде из 'data/preprocessing_loaded_table.csv'

# Кодировка информации об осадках из колонки WW
def true_weather_WW_replace(ww):
    if ww=='нет осадков':
        return 0
    elif ww=='слабый дождь':
        return 1
    elif (ww=='сильный дождь') or (ww=='снег'):
        return 2
    else:
        return 3

# Вычисление Timestamp из даты и времени
def row_plus_hours_to_index(row):
    return row['date'] + pd.to_timedelta(row['time'] , 'H')

# Функция для сдвига на сутки (в скачанном датасете разбивка по 30 мин, поэтому timeshift=48)
def shift_features_fact(df, timeshift=48):
    list_fact_columns=list(df.columns)
    list_fact_columns.remove('date_tw')
    new_df = df.copy()
    for column in list_fact_columns:
        new_df[column] = new_df[column].shift(timeshift)

    return new_df

In [ ]:
# Функция для вычисления метрики mae по дням из почасовых массивов данных

def mae_day(y_true, y_pred):
    y_true_copy = pd.DataFrame(y_true).reset_index(drop=True)
    y_true_copy['day'] = y_true_copy.index // 24
    y_true_grouped = y_true_copy.groupby(by='day').sum()   
    y_pred_copy = pd.DataFrame(y_pred).reset_index(drop=True)
    y_pred_copy['day'] = y_pred_copy.index // 24
    y_pred_grouped = y_pred_copy.groupby(by='day').sum()
    
    return mean_absolute_error(y_true_grouped, y_pred_grouped)
# Функция для вычисления метрик по дням из почасовых массивов данных

def metrics_hour(y_true, y_pred):

    
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mae, mape, r2

#### 1.5 Чтение файлов с данными
Данные объединяются в один датасет

In [ ]:
# читаем исходные датасеты и складываем в один
train_ds = pd.read_csv('data/train_dataset.csv')
test_ds = pd.read_csv('data/test_dataset.csv')
train_ds = pd.concat([train_ds, test_ds])

# запоминаем дату начала тестовых данных, потом также поступим и с закрытым датасетом
open_test_begin = pd.to_datetime(test_ds['date']).min()
open_test_end = pd.to_datetime(test_ds['date']).max() + pd.to_timedelta(1,'d')
print('начало открытого теста:', open_test_begin, '    конец открытого теста:', open_test_end)

#### 1.6 Формирование колонок с производными от даты

In [ ]:
# преобразуем дату и делаем из нее колонки
train_ds['date'] = pd.to_datetime(train_ds['date'])
train_ds['year'] = train_ds['date'].dt.year
train_ds['month'] = train_ds['date'].dt.month
train_ds['day_of_week'] = train_ds['date'].dt.dayofweek
train_ds['day'] = train_ds['date'].dt.day
train_ds['day_of_year'] = train_ds['date'].dt.dayofyear

In [ ]:
train_ds.sample(5)

#### 1.7 Подгрузка Auggumentaci в праздниках


df_holidays = pd.read_csv('data/holidays_aug.csv')
df_holidays['date'] = pd.to_datetime(df_holidays['date'])
# Добавление данных о праздниках из файла 'data/holidays.csv'


print('размер DS', train_ds.shape, 'дубликатов - ', train_ds.duplicated().sum())
# Assuming df_holidays and train_ds are your dataframes
train_ds = pd.merge(train_ds, df_holidays, on='date', how='left')
print('размер DS', train_ds.shape, 'дубликатов - ', train_ds.duplicated().sum())
# Fill NaN values with 0
train_ds['holidays'].fillna(0, inplace=True)
train_ds['preholidays'].fillna(0, inplace=True)

# Convert to int
train_ds['holidays'] = train_ds['holidays'].astype(int)
train_ds['preholidays'] = train_ds['preholidays'].astype(int)
print('размер DS', train_ds.shape, 'дубликатов - ', train_ds.duplicated().sum())

#### 1.7.1 Подгрузка данных о праздниках на весь DS праздниках

In [ ]:

df_holidays_true = pd.read_csv('data/holidays_true.csv')
df_holidays_true['date'] = pd.to_datetime(df_holidays_true['date'])
# Добавление данных о праздниках из файла 'data/holidays.csv'
print('размер DS', train_ds.shape, 'дубликатов - ', train_ds.duplicated().sum())
# Assuming df_holidays and train_ds are your dataframes
train_ds = pd.merge(train_ds, df_holidays_true, on='date', how='left')

# Fill NaN values with 0
train_ds['holidays_true'].fillna(0, inplace=True)
train_ds['preholidays_true'].fillna(0, inplace=True)

# Convert to int
train_ds['holidays_true'] = train_ds['holidays_true'].astype(int)
train_ds['preholidays_true'] = train_ds['preholidays_true'].astype(int)
print('размер DS', train_ds.shape, 'дубликатов - ', train_ds.duplicated().sum())

In [ ]:
train_ds.info()

#### 1.8 Формирование колонок со значением целевого признака в предыдущие дни

In [ ]:
# Добавление колонок с временными лагами

# создаем столбец 'temp_last_day'
train_ds['temp_last_day'] = train_ds['temp'].shift(24)
print('размер DS', train_ds.shape, 'дубликатов - ', train_ds.duplicated().sum())
# заполняем пропущенные значения в 'temp_last_day'
train_ds['temp_last_day'].fillna(method='bfill', inplace=True)

# создаем столбцы с временными лагами для 'target'
lags = [24, 48, 72, 7*24, 14*24]
for lag in lags:
    train_ds[f'target_lag_{lag}'] = train_ds['target'].shift(lag)

# заполняем пропущенные значения в столбцах с лагами
for lag in lags:
    train_ds[f'target_lag_{lag}'].fillna(0, inplace=True)

print('размер DS', train_ds.shape, 'дубликатов - ', train_ds.duplicated().sum())

#### 1.9 Формирование колонок с ВВП и данными о погоде посредством ранее описанных функций

In [ ]:
# применяем функцию добавления ВВП
train_ds = add_vvp2(train_ds)

# Расшифровка прогноза в колонке 'weather_pred'
train_ds = fill_weather_columns(train_ds)


# Читаем файл с архивом фактической погоды
df_true_weather = pd.read_csv('data/preprocessing_loaded_table.csv')
display(df_true_weather)

# Форматируем колонки
df_true_weather['WW'] = df_true_weather['WW'].apply(true_weather_WW_replace)
df_true_weather['date'] = pd.to_datetime(df_true_weather['date'])
df_true_weather = df_true_weather.rename(columns={'date':'date_tw'})
# Применяем сдвиг на сутки, чтобы не заглядывать в будущее
df_true_weather = shift_features_fact(df_true_weather)
# Добавляем в датасет
train_ds['date_hours'] = train_ds.apply(row_plus_hours_to_index, axis=1)
train_ds = train_ds.merge(df_true_weather, left_on='date_hours', right_on='date_tw')
train_ds = train_ds.drop(['date_hours', 'date_tw'], axis=1)

###  Новые фичи

from add_new_data import (add_euro,
                            add_light_work,
                            add_stock,
                            add_ipp_mm,
                            add_ipp_yy,)

#### 1.9.1 Euro

FEATURES = 'add_euro'
train_ds = add_euro(train_ds)



#### 1.9.2 Stock

FEATURES = 'add_stock'
train_ds = add_stock(train_ds)



#### 1.9.3 IPP Month

FEATURES = 'add_ipp_mm'
train_ds = add_ipp_mm(train_ds)



#### 1.9.4 IPP Year

FEATURES = 'add_ipp_yy'
train_ds = add_ipp_yy(train_ds)



#### 1.9.5 Light work

FEATURES = 'add_light_work'
train_ds = add_light_work(train_ds)



#### 1.9.6 All

FEATURES = 'add_all'

train_ds = add_euro(train_ds)
train_ds = add_stock(train_ds)
train_ds = add_ipp_mm(train_ds)
train_ds = add_ipp_yy(train_ds)
train_ds = add_light_work(train_ds)



#### 1.10 Демонстрация сформированного датасета

In [ ]:
# Итоговый набор колонок
train_ds.columns

In [ ]:
train_ds.head()

#### 1.11.1 Добавление среднего за час предыдущего дня

In [ ]:

def mean_evening(values, evening=19):
    return values[evening:].mean()

evening_slices = [0, 19, 22]
    
for evening_slice in evening_slices:
    train_ds[['last_evening_avg_target_'+str(evening_slice), 'last_evening_avg_temp_'+str(evening_slice)]] = \
        train_ds[['date', 'target', 'temp']].groupby(by='date').transform(mean_evening, evening=evening_slice).shift(24)

In [ ]:
train_ds.columns

#### 1.11 Исключение лишних колонок

In [ ]:
# Отбираем признаки. Все лишние колонки здесь отбрасываем, кроме 'date', которую уберем позже 

feature_cols = list(train_ds.columns)

# выбрасываем взгляд в прошлое и расшифрованную погоду
drop_list = ['target', 'day_of_year', 'weather_pred', 'weather_fact', 'temp']

# выбрасываем признаки, найденные процедурно в процессе оптимизации
# КОМАНДЕ: здесь можно добавлять признаки на выброс с целью оптимизации
drop_list = drop_list + ['target_lag_48', 'target_lag_168'] #, 'temp_pred'] #, 'target_lag_336'] 

for name in drop_list:
    feature_cols.remove(name)

# Итоговый список признаков
feature_cols

#### 1.11.2 Добавление среднего за час предыдущего дня

In [ ]:
FEATURE_WINDOW_SIZE = 24
feature_cols_no_date = feature_cols.copy()
feature_cols_no_date.remove('date')


for lag in range(1,FEATURE_WINDOW_SIZE):
    for column in feature_cols_no_date:
        train_ds[column+'_'+str(lag)] = train_ds[column].shift(lag)

#### 1.11.2 Добавление лагов за час 

In [ ]:
target_lags=[1, 5, 9]

for lag in target_lags:
    train_ds['target_'+str(lag)] = train_ds['target'].shift(lag).where(train_ds['time']<lag, np.NaN)
    train_ds['temp_'+str(lag)] = train_ds['temp'].shift(lag).where(train_ds['time']<lag, np.NaN)

In [ ]:
feature_cols = list(train_ds.columns)
for name in drop_list:
    feature_cols.remove(name)

# Итоговый список признаков
feature_cols

#### 1.11.13 Аугументации

#### 1.12 Выделение наборов данных для обучения, валидации и тестирования

Выделялось два набора данных для обучения и валидации:
1. Обучение на данных с 2019 по 2021 с валидацией на 2022
2. Обучение на данных с 2019 по 2022 с валидацией на первом квартале 2023

Первый набор позволяет оценить влияние сезонности на обучение и предсказания, второй позволяет обучить модель на большем объеме данных и на более актуальных данных.

In [ ]:
train_ds['month']

In [ ]:
# Формируем набор датасетов для обучения и проверки
train_ds = train_ds.sort_values(by=['date', 'time']).reset_index(drop=True)
features = train_ds[feature_cols]
target = train_ds['target']

# Функция для выделения временных интервалов из таблиц признаков и целей
# на этом этапе отбрасываем колонку 'date'
def features_interval(features, target, date1, date2):
    features_interval = features[ (features['date']>=date1) & (features['date']<date2) ]
    target_interval = target[features_interval.index]
    features_interval = features_interval.drop('date', axis=1)
    return features_interval, target_interval


# для проверки на тестовой выборке будем учиться на всем тренировочном датасете

features_open_test, target_open_test = features_interval(features, target, open_test_begin, open_test_end)


features_open_test_sum, target_open_test_sum = features_interval(features, target, '2023-06-01', open_test_end)



FEATURES = 'test_summer_fulltest'



### 2. Обучение моделей

В настоящей работе обучается модель LightGBM

#### 2.1 Гиперпараметры
Были подобраны следующие значения гиперпараметров:

In [ ]:
params = {'num_leaves':15, 'learning_rate':0.02, 'feature_fraction':1, 'num_iterations':NUM_ITERATIONS, 'random_state':random_state, 'objective':'regression_l1', 'n_jobs':-1}

## XGBoost build

In [ ]:
#features_train, target_train
#features_valid, target_valid

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

### 4. Проверка метрик на тестовом датасете

#### 4.1 LGBM

In [ ]:
features_all_train.columns

In [ ]:
drop_list = [#'preholidays',
            #'has_rain_probability', 'W', 'E',
            #'holidays' 
            ]
feat_lgbm_train = features_all_train.drop(columns=drop_list)
feat_lgbm_test = features_open_test.drop(columns=drop_list)
feat_lgbm_train.columns, feat_lgbm_test.columns

In [ ]:
# Проверка метрики лучшей модели на тестовом датасете
# Здесь обучаем на всем тренировочном датасете

lgbm_model_all_train = lgb.Booster(model_file='models/lgb_model.txt')

l_predict_train = lgbm_model_all_train.predict(feat_lgbm_train)
l_predict_test = lgbm_model_all_train.predict(feat_lgbm_test)

mae_train, mape_train, r2_train = metrics_hour(target_all_train, l_predict_train)
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test, l_predict_test)

results = pd.DataFrame([[f'тренировочная LGBM {FEATURES}', mae_train, mape_train, r2_train], [f'тестовая LGBM {FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))

In [ ]:
# Проверка метрики лучшей модели на тестовом датасете
# Здесь обучаем на всем тренировочном датасете




l_predict_test = lgbm_model_all_train.predict(features_open_test_sum)


mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test_sum, l_predict_test)

results = pd.concat([results,
pd.DataFrame([ [f'тестовая LGBM_only summer {FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))
 ])

In [ ]:
results

#### 4.2 XGBoost

In [ ]:
features_all_train.columns

In [ ]:
drop_list = ['preholidays_true',
            'has_rain_probability', 
            #'holidays', 'preholidays',
             'W', 'E'
            ]
feat_xgb_train = features_all_train.drop(columns=drop_list)
feat_xgb_test = features_open_test.drop(columns=drop_list)
feat_xgb_train.columns, feat_xgb_test.columns

In [ ]:
drop_list = ['preholidays_true',
            #'has_rain_probability', 
            # 'W', 'E'
            ]
n_values = range(1, 24)
preholidays = ['preholidays_true_{}'.format(n) for n in n_values]
#has_rain = ['has_rain_probability_{}'.format(n) for n in n_values]
#W_wind = ['W_{}'.format(n) for n in n_values]
#E_wind = ['E_{}'.format(n) for n in n_values]

drop_list = drop_list + preholidays #+ has_rain + W_wind + E_wind

feat_xgb_train = features_all_train.drop(columns=drop_list)
feat_xgb_test = features_open_test.drop(columns=drop_list)
feat_xgb_train.columns, feat_xgb_test.columns

In [ ]:
# Проверка метрики лучшей модели на тестовом датасете
# Здесь обучаем на всем тренировочном датасете


xgb_model_all_train = XGBRegressor()

# загружаем модель из файла
xgb_model_all_train.load_model('models/xgb_modelbase feature_3.json')

xgb_predict_test = xgb_model_all_train.predict(feat_xgb_test)
xgb_predict_train = xgb_model_all_train.predict(feat_xgb_train)

mae_train, mape_train, r2_train = metrics_hour(target_all_train, xgb_predict_train )
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test, xgb_predict_test )

results = pd.concat([results,
pd.DataFrame([[f'тренировочная XGB {FEATURES}', mae_train, mape_train, r2_train], [f'тестовая XGB {FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))
 ])


In [ ]:
xgb_predict_test = xgb_model_all_train.predict(feat_xgb_test)


mae_train, mape_train, r2_train = metrics_hour(target_all_train, xgb_predict_train )
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test, xgb_predict_test )

results = pd.concat([results,
pd.DataFrame([[f'тренировочная XGB {FEATURES}', mae_train, mape_train, r2_train], [f'тестовая XGB {FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))
 ])


results.to_csv(f'results_LGBM_XGBoost_{FEATURES}_no_h')

In [ ]:
results

### 5. Ensemble

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNet
from lightgbm import LGBMRegressor
from sklearn.ensemble import StackingRegressor

import optuna
from optuna.samplers import TPESampler

In [ ]:
results_ensemble = pd.DataFrame(columns=('Выборка', 'MAE', 'MAPE', 'R2'))

### Simple Ensemble

In [ ]:
predict_simple_ensemble_train = (xgb_predict_train + l_predict_train)/2
predict_simple_ensemble_test = (xgb_predict_test + l_predict_test)/2



In [ ]:
mae_train, mape_train, r2_train = metrics_hour(target_all_train, predict_simple_ensemble_train)
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_open_test, predict_simple_ensemble_test)

results_ensemble = pd.concat([results_ensemble,
pd.DataFrame([[f'тренировочная simple_ensemble  {FEATURES}', mae_train, mape_train, r2_train], [f'тестовая simple_ensemble {FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))
 ])

In [ ]:
display(results_ensemble)
results

In [ ]:
mae_day(target_open_test, predict_simple_ensemble_test)

import os

# определите путь к папке, которую вы хотите создать
folder_path = "models"

# проверьте, существует ли уже папка
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

lgbm_model_all_train.booster_.save_model('models/lgb_model.txt')
xgb_model.save_model('models/xgb_model.json')

In [ ]:
results.to_csv(f'models/results_LGBM_XGBoost_simple_ensemble_{FEATURES}')